In [ ]:
# pip install requests
# pip install pandas
# pip install bs4

In [ ]:
# Product A = 12 pack maggie

import requests
url='http://web.archive.org/cdx/search/cdx'
target='www.amazon.in/dp/B07B4KQRZG*'
try :
  response=requests.get(url,params={'url':target,
                                    'output':'json',
                                    'filter':'statuscode:200',
                                    'fl':'original,timestamp',
                                    'collapse':'timestamp:6'},timeout=30)
  response.raise_for_status()
  raw_data=response.json()

except requests.exceptions.RequestException as e:
  print(e)

HTTPConnectionPool(host='web.archive.org', port=80): Max retries exceeded with url: /cdx/search/cdx?url=www.amazon.in%2Fdp%2FB07B4KQRZG%2A&output=json&filter=statuscode%3A200&fl=original%2Ctimestamp&collapse=timestamp%3A6 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x790def65e270>, 'Connection to web.archive.org timed out. (connect timeout=30)'))


In [ ]:
import pandas as pd
print(type(raw_data))
df=pd.DataFrame(raw_data[1:],columns=raw_data[0])
print(df.head())
print(df.tail())
len(df)

NameError: name 'raw_data' is not defined

In [ ]:
df['timestamp-datetime']=pd.to_datetime(df['timestamp'].to_list(),format='%Y%m%d%H%M%S').date
df.drop_duplicates(subset=['timestamp-datetime'],inplace=True)
df

In [ ]:
# https://web.archive.org/web/ + timestamp + / + original_url
# https://web.archive.org/web/20201105181717/https://www.amazon.in/dp/B07B4KQRZG

df.head()
df['webpages']='https://web.archive.org/web/'+df['timestamp']+'/'+df['original']
df.head()

In [ ]:
df.sort_values(by=['timestamp-datetime'],inplace=True)
df.reset_index(drop=True,inplace=True)
df

In [ ]:
# PRACTICE !!
# testing_url=df['webpages'].iloc[-10]
# response=requests.get(testing_url,timeout=10)
# response.raise_for_status()
# html=response.text

In [ ]:
# PRACTICE !!
# import bs4
# from bs4 import BeautifulSoup
# soup=BeautifulSoup(html,'html.parser')
# soup.prettify()
# # soup.title.string
# title=soup.find(id='productTitle').get_text().strip()
# title
# price=soup.find('span',attrs={'class':"a-price-whole"}).get_text().strip()

In [ ]:

# PRACTICE !!
# testing_url2=df['webpages'].iloc[-300]
# import bs4
# from bs4 import BeautifulSoup
# response=requests.get(testing_url2)
# html=response.content
# soup=BeautifulSoup(html)
# print(soup.find(id='productTitle'))
# print(soup.find('span',{'class':'a-price-whole'}))

# result={}
# result['title_col']=12
# result
# df['webpages'].iloc[0]

In [ ]:
def parse_page(webpage_link):
  import time
  import re
  from bs4 import BeautifulSoup
  result={'title':None,'price':None}
  time.sleep(10)
  try:
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response=requests.get(webpage_link,timeout=20,headers=headers)
    html=response.content
    soup=BeautifulSoup(html,'html.parser')

    if soup.find(id='productTitle') is None:
      result['title']=None
    else:
      webpage_title=soup.find(id='productTitle').get_text().strip()
      result['title']=webpage_title


    if soup.find('span',{'class':"a-price-whole"}) is not None:
      webpage_price=soup.find('span',{'class':"a-price-whole"}).get_text().strip()

    elif soup.find(id='priceblock_dealprice') is not None:
      webpage_price=soup.find(id='priceblock_dealprice').get_text().strip()

    elif soup.find(id='priceblock_ourprice') is not None:
      webpage_price=soup.find(id='priceblock_ourprice').get_text().strip()

    else:
      webpage_price=None

    if webpage_price is not None:
      webpage_final_price=re.sub(r'[^\d.]','',webpage_price)
      result['price']=float(webpage_final_price)

    else:
      result['price']=webpage_price

    return result

  except Exception as e:
    print(e)



In [ ]:
## scraped_data=df['webpages'].apply(parse_page)
# parse_page(df['webpages'].iloc[:20])


In [ ]:
# scraped_data=scraped_data.apply(pd.Series)
# scraped_data.to_csv('scraped_data')
scraped_data=pd.read_csv('/content/scraped_data')
scraped_data=scraped_data.drop("Unnamed: 0",axis=1)

In [ ]:
maggie_raw_df=pd.concat([df,scraped_data],axis=1)
maggie_raw_df=maggie_raw_df.dropna()

In [ ]:
maggie_raw_df.head()

In [ ]:
test_text=maggie_raw_df['title'][0]
print(test_text)
# import re
# weight=re.search(r'(\d*\.?\d+)\s*(g|kg|gm|gram|ml|l)',test_text)
# print(weight.group(1))

In [ ]:
def extract_product_details(title_text):
  import re
  result={}
  if title_text is not None:

    no_of_packs=1
    grps=re.search(r'Pack of\s+(\d+)',title_text,flags=re.IGNORECASE)
    if grps:
      no_of_packs=int(grps.group(1))
      result['no_of_packs']=no_of_packs

    grps2=re.search(r'(\d*\.?\d+)\s*(g|kg|gm|gram|ml|l)',title_text,flags=re.IGNORECASE)
    if grps2:
      weight_per_pack=float(grps2.group(1))
      unit=grps2.group(2)
      result['weight_per_pack']=weight_per_pack
      result['unit']=unit
      if weight_per_pack>100 :
        total_weight=weight_per_pack
      else:
        total_weight=weight_per_pack*no_of_packs

      result['total_weight']=total_weight

    return result
  else :
    result={'weight_per_pack':None,'unit':None,'no_of_packs':None,'total_weight':None}
    return result

In [ ]:
# extract_product_details(test_text)

information_df=maggie_raw_df['title'].apply(extract_product_details).apply(pd.Series)
print(information_df['unit'].unique())
all_df=pd.concat([maggie_raw_df,information_df],axis=1)


In [ ]:
all_df['price_per_100g']=(all_df['price']/all_df['total_weight'])*100
all_df.info()

In [ ]:
needed_df=all_df[['timestamp-datetime','price','weight_per_pack','no_of_packs','total_weight','price_per_100g']]
needed_df['timestamp-datetime']=needed_df['timestamp-datetime'].apply(pd.to_datetime)
needed_df=needed_df.dropna().reset_index(drop=True)
needed_df

In [ ]:
import matplotlib as plt
import seaborn as sns
needed_df.plot(kind='scatter',
               x='timestamp-datetime',
               y='price_per_100g',
               title='The Real Cost of Maggi (2020-2024)',
               )

In [ ]:
#Moving average
needed_df['moving_average']=needed_df['price_per_100g'].rolling(10).mean()
needed_df

In [ ]:
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(8,5))
plt.style.use('_mpl-gallery')
# plt.plot(needed_df['timestamp-datetime'],needed_df['inflation_index/price_per_100g'])
plt.scatter(x=needed_df['timestamp-datetime'],
            y=needed_df['price_per_100g'],
            alpha=0.5)
plt.plot(needed_df['timestamp-datetime'],
         needed_df['moving_average'],color='red')
plt.legend(['scatterplot','moving average'])
plt.title('Moving average of inflation since 2020-25')
plt.show()


In [ ]:
fig=plt.figure(figsize=(8,5))
plt.style.use('_mpl-gallery')
# plt.plot(needed_df['timestamp-datetime'],needed_df['inflation_index/price_per_100g'])
plt.plot(needed_df['timestamp-datetime'],
            needed_df['total_weight'],
            color='green',lw=3)
# plt.plot(needed_df['timestamp-datetime'],
#          needed_df['moving_average'],color='red')
plt.legend(['weight is constant at 840g'])
plt.title('Date Vs Weight')
plt.savefig('Date Vs Weight')

In [ ]:
#E70023
#FFF01D
import seaborn as sns
plt.figure(figsize=(8,5),dpi=300)
sns.scatterplot(needed_df,
                x='timestamp-datetime',
                y='price_per_100g',
                color='grey',
                alpha=0.4,
                label='Raw Data Points')
sns.lineplot(needed_df,
             x='timestamp-datetime',
             y='moving_average',
             color='#E70023',
             lw=2,
             label='10-Point Moving Average'
             )

plt.xlabel('')
plt.ylabel('Price per 100g in Rupees(₹)',fontsize=12)
plt.grid(True,alpha=0.5,color='#e0e0e0',ls='dotted')
plt.title('Maggi Unit Price tracks Inflation (2020-2025)',fontsize=19,fontweight='bold')
sns.despine()
plt.savefig('Maggie unit price tracks inflation(2020-2025)',bbox_inches='tight')

In [ ]:
# --- CHART 1: FINAL POLISHED VERSION ---
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

# Set style
sns.set_theme(style="white", rc={"axes.grid": True, "grid.color": "#e0e0e0", "grid.linestyle": ":"})

plt.figure(figsize=(10, 6), dpi=300) # Standard Landscape for LinkedIn Slides

# 1. Plot Raw Data (Background Noise)
sns.scatterplot(
    data=needed_df,
    x='timestamp-datetime',
    y='price_per_100g',
    color='grey',
    alpha=0.3, # Made slightly more transparent to let the red line pop
    s=60,
    label='Raw Data Points'
)

# 2. Plot Moving Average ( The Signal)
sns.lineplot(
    data=needed_df,
    x='timestamp-datetime',
    y='moving_average',
    color='#D32F2F', # Deep Maggi Red
    linewidth=2.5,
    label='10-Point Moving Average'
)

# 3. Professional Labels
plt.ylabel('Price per 100g (₹)', fontsize=12, fontweight='bold', labelpad=10)
plt.xlabel('') # Remove "timestamp-datetime". The years speak for themselves.

# 4. The Title (Clean, no quotes, padded)
plt.title('Maggi Unit Price tracks Inflation (2020-2025)',
          fontsize=18,
          fontweight='bold',
          color='#333333', # Dark grey is classier than pure black
          pad=20) # Pushes title up

# 5. Axis Formatting (Clean Years)
ax = plt.gca()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax.xaxis.set_major_locator(mdates.YearLocator())

# 6. Remove the Box (Spines)
sns.despine(left=False, bottom=False) # Keep L and Bottom, remove Top and Right

# 7. Legend adjustment
plt.legend(frameon=False, loc='upper left')

# Save
plt.tight_layout()
plt.savefig('Maggi_Price_Analysis_Final.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- CHART 2: WEIGHT STABILITY ---
plt.figure(figsize=(10, 6), dpi=300)

# 1. The Step Chart (Crucial for inventory/weight data)
# 'where=post' means the line stays flat until the change actually happens
plt.step(
    needed_df['timestamp-datetime'],
    needed_df['total_weight'],
    where='post',
    color='#FBC02D', # Maggi Noodle Gold
    linewidth=3
)

# 2. The "Honesty" Scale
# Force Y-axis to start at 0.
# If you don't, it will zoom in on 840g and look like a flat line in space.
# Showing 0 proves the weight is substantial.
plt.ylim(0, 1000)

# 3. Annotation (The Story)
# Since the line is flat, we tell the user WHY it's flat.
plt.text(
    x=mdates.date2num(needed_df['timestamp-datetime'].iloc[int(len(needed_df)/2)]),
    y=500, # Position text in the empty space below the line
    s=" NO SHRINKFLATION\nWeight constant at 840g",
    ha='center',
    fontsize=14,
    fontweight='bold',
    color='#333333',
    bbox=dict(boxstyle="round,pad=0.5", fc="white", ec="#FBC02D", lw=2)
)

# 4. Polish
plt.title('Maggi Pack Weight Stability (2020-2025)', fontsize=18, fontweight='bold', pad=20, color='#333333')
plt.ylabel('Total Weight (grams)', fontsize=12, fontweight='bold')
plt.xlabel('')

# Format Dates
ax = plt.gca()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax.xaxis.set_major_locator(mdates.YearLocator())

sns.despine()
plt.tight_layout()
plt.savefig('Maggi_Weight_Stability.png', bbox_inches='tight')
plt.show()

In [ ]:
## Product B Toblerone Milk Chocolate Bar

data = [
    {'date': '2016-01-01', 'weight': '170g', 'price(GBP)': 1.00, 'event': 'Before shrink'},
    {'date': '2016-02-01', 'weight': '170g', 'price(GBP)': 1.00, 'event': 'Before shrink'},
    {'date': '2016-03-01', 'weight': '170g', 'price(GBP)': 1.00, 'event': 'Before shrink'},
    {'date': '2016-04-01', 'weight': '170g', 'price(GBP)': 1.00, 'event': 'Before shrink'},
    {'date': '2016-05-01', 'weight': '170g', 'price(GBP)': 1.00, 'event': 'Before shrink'},
    {'date': '2016-06-01', 'weight': '170g', 'price(GBP)': 1.00, 'event': 'Before shrink'},
    {'date': '2016-07-01', 'weight': '170g', 'price(GBP)': 1.00, 'event': 'Before shrink'},
    {'date': '2016-08-01', 'weight': '170g', 'price(GBP)': 1.00, 'event': 'Before shrink'},
    {'date': '2016-09-01', 'weight': '170g', 'price(GBP)': 1.00, 'event': 'Before shrink'},
    {'date': '2016-10-01', 'weight': '170g', 'price(GBP)': 1.00, 'event': 'Before shrink'},
    {'date': '2016-11-01', 'weight': '150g', 'price(GBP)': 1.00, 'event': 'SHRINK EVENT (170g -> 150g)'},
    {'date': '2016-12-01', 'weight': '150g', 'price(GBP)': 1.00, 'event': 'After shrink'},
    {'date': '2017-01-01', 'weight': '150g', 'price(GBP)': 1.00, 'event': 'After shrink'},
    {'date': '2017-02-01', 'weight': '150g', 'price(GBP)': 1.00, 'event': 'After shrink'},
    {'date': '2017-03-01', 'weight': '150g', 'price(GBP)': 1.00, 'event': 'After shrink'},
    {'date': '2017-04-01', 'weight': '150g', 'price(GBP)': 1.00, 'event': 'After shrink'},
    {'date': '2017-05-01', 'weight': '150g', 'price(GBP)': 1.00, 'event': 'After shrink'},
    {'date': '2017-06-01', 'weight': '150g', 'price(GBP)': 1.00, 'event': 'After shrink'},
    {'date': '2017-07-01', 'weight': '150g', 'price(GBP)': 1.00, 'event': 'After shrink'},
    {'date': '2017-08-01', 'weight': '150g', 'price(GBP)': 1.00, 'event': 'After shrink'},
]

In [ ]:
import pandas as pd
df=pd.DataFrame(data)

import re
# re_str=df['weight'][1]
# re.sub(r'\D','',re_str)
df['weight']=df['weight'].apply(lambda x:re.sub(r'\D','',x)).astype(float)
df['date']=df['date'].apply(pd.to_datetime)
df['price_per_100g']=(df['price(GBP)']/df['weight'])*100
df


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from matplotlib.lines import lineMarkers
plt.figure(figsize=(10,8),dpi=300)

# Scatterplot
sns.scatterplot(df,
                x='date',y='price_per_100g',
                color='grey',
                alpha=0.3,
                edgecolor='#696969'
                )

# lineplot
sns.lineplot(df,x='date',y='price_per_100g',color='#A30000',alpha=0.8,lw=1.5)

# points
plt.plot(pd.to_datetime('2016-10-01'),0.588235,marker='o',color='grey')
plt.plot(pd.to_datetime('2016-11-01'),0.666667,marker='o',color='grey')

# annotation
bbox = dict(boxstyle="round", fc="0.8")
arrowprops=dict(facecolor='black', arrowstyle='->')
plt.annotate('(10-2016,170g)',xy=(pd.to_datetime('2016-10-01'),0.588235),xytext=(pd.to_datetime('2016-07-01'),0.583),bbox=bbox,arrowprops=arrowprops)
plt.annotate('(11-2016,150g)',xy=(pd.to_datetime('2016-11-01'),0.666667),xytext=(pd.to_datetime('2016-11-08'),0.67),bbox=bbox,arrowprops=arrowprops)
plt.annotate('Weight dropped \n 170g → 150g',xy=(pd.to_datetime('2016-9-1'),0.627),fontweight='bold',bbox=bbox)

# yaxis tickers
from matplotlib.ticker import FuncFormatter
ax=plt.gca()
ax.yaxis.set_major_formatter(FuncFormatter(lambda x,pos: f'£{x:.2f}'))


# shades
plt.axvline(pd.to_datetime('2016-10-15'),ls='--',color='black',alpha=0.3)
plt.axvspan(xmin=pd.to_datetime('2016-01-01'),xmax=pd.to_datetime('2016-10-15'),color='green',alpha=.2)
plt.axvspan(xmin=pd.to_datetime('2016-10-15'),xmax=pd.to_datetime('2017-08-01'),color='red',alpha=.2)

# title,limits,grid,label
plt.ylim(0.58,0.68)
plt.ylabel('Price Per 100g',fontsize=12,labelpad=13)
plt.xlabel('')
plt.title("Toblerone's Hidden Inflation: Maintaining the £1 Price Point",pad=13,fontweight='bold',fontsize=15,loc='left')
plt.grid(color='#D3D3D3',linestyle='dotted',alpha=0.6)

# figtext
plt.figtext(x=.4,y=.01,s='Analysis by KartikSavaliya',color='#333333')

# spines
sns.despine()

# save
plt.savefig('Shrinkflation Graph of Product B')
